<a href="https://colab.research.google.com/github/jays0n/Projeto-CadKlassmatt/blob/main/RPAKlassmatt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto RPA Klassmatt**

In [1]:
#Installs
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,326 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-bac

In [2]:
#Imports:
import time,os
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from html import *
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
from datetime import *

In [3]:
#@title Montagem
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Constantes:
url_site=r"https://login.klassmatt.com.br/Login.aspx"
user_login=""
password_login=""

In [5]:
#Support Functions:
def Timer(seconds):
  sleep(seconds)

def Go(driver_object):
  try:
    driver_object.get(url_site)
  except Exception as err:
    print("Erro no browser ao tentar carregar a url:"+url_site + "\nVerifique sua conexao.")

def Login(driver_object):
  Timer(3)
  analyse=Wait_Until(driver_object,By.ID,'txtUsuario')
  scripts=["document.getElementById('txtUsuario').value='{0}'".format(user_login),"document.getElementById('txtSenha').value='{0}'".format(password_login),"document.getElementById('cmdEntrar').click()"]

  for script in scripts:
    try:
      driver_object.execute_script(script)
      Timer(1)
    except Exception as err:
      pass

def Quit(driver_object):
  try:
    driver_object.Close()
  except:
    pass

def Wait_Until(driver_object,by,tag,expected_condition=None,seconds=1,max_attempts=10):
  attempts=max_attempts
  expected_condition_try=EC.presence_of_element_located if expected_condition is None else expected_condition
  while attempts>0:
      try:
          WebDriverWait(driver_object,seconds).until(expected_condition_try((by,tag)))
          return True
      except Exception as err:
          pass
      Timer(1)
      attempts-=1
  print("Erro no procedimento 'Wait' apos {0} tentativas de encontrar '{1}' sem sucesso!".format(max_attempts,tag))
  return False

def Close_Alerts(driver_object):
  k=0
  msg_err=None
  while k<10:
    try:
      driver_object.switch_to.alert().dismiss()
      driver_object.switch_to.default_content()
      break
    except Exception as err:
      msg_err=str(err)
      pass
    k+=1
  if msg_err!=None:
    print("Erro ao fechar o 'Alert':\n",msg_err)

def Switch_By_Partial_Url(driver_object,partial_url=[]):
  url=driver_object.current_url

  for hwnd in driver_object.window_handles:
    driver_object.switch_to.window(hwnd)
    purl=driver_object.current_url
    truely=False
    if str(type(partial_url)).lower() == 'list':
      truely=True
      for u in partial_url:
        if purl.lower().find(u.lower())<=0:
          truely=False
        if not truely:
          break
    elif str(type(partial_url)).lower() == 'str':
      if purl.lower().find(partial_url.lower())<=0:
        truely=True

    if truely:
      print("Mudando para -> Pop up ...")
      break
    else:
      driver_object.switch_to.default_content()




In [6]:
#Chrome Initialize:
def Create_Driver():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  return driver

In [7]:
def Start_Registration(driver_object,title,description):
  print("Start Registration ...")
  scripts=["__doPostBack('ctl00$Body$rptSolicitacaoItemNovo$ctl00$lkbutMenu','')","document.getElementById('butNotFoundItem').click()"]
  Timer(2)
  for script in scripts:
    driver_object.execute_script(script)
    Timer(2)
  driver_object.execute_script("document.getElementById('txtNome').value='"+title+"'")
  Timer(1)
  driver_object.execute_script("document.getElementById('txtDescricao').value='"+description+"'")
  Timer(2)
  driver_object.execute_script("document.getElementById('butContinuar').click()")
  Timer (2)

def Set_Types(driver_object,txtTipo,txtUnidadeMedida):
  print("Start Setting Types ...")
  scripts=["document.getElementById('txtTipo').value='"+txtTipo+"'","document.getElementById('txtUnidadeMedida').value='"+txtUnidadeMedida+"'"]
  for script in scripts:
    driver_object.execute_script(script)
    Timer(2)

def Set_AS_Urgent(driver_object,urgent=False):
  print("Verifing Urgencie ...")
  if urgent:
    driver_object.execute_script("__doPostBack('ctl00$Body$dlPrioridade$ctl02$LinkButton2','')")
    Timer(2)

def Set_Classification(driver_object,grupo,subgrupo,condicao_transacao):
  print("Setting classification ...")
  scripts=["__doPostBack('ctl00$Body$dlTab$ctl02$lbutMenu','')","document.getElementById('ctl00_Body_tabCategorias_ucCategorias1_gvCategorias_ctl02_txtCat').value='{0}'".format(grupo),
           "document.getElementById('ctl00_Body_tabCategorias_ucCategorias1_gvCategorias_ctl03_txtCat').value='{0}'".format(subgrupo),
           "document.getElementById('ctl00_Body_tabCategorias_ucCategorias1_gvCategorias_ctl04_txtCat').value='{0}'".format(condicao_transacao)]

  k=0
  for script in scripts:
    driver_object.execute_script(script)
    if k==0:
      Timer(2)
    else:
      Timer(1)
    k+=1
  Timer(2)

def Set_Midia(driver_object,midia):
  print("setting midia up ...")
  Timer(2)
  idsin=0
  try:
    url=driver_object.current_url
    idsin=url.lower().find('idsin=')+6
    limit=idsin+6
    idsin=int(url[idsin:limit])
    driver_object.execute_script("javascript:{OpenWindowsWHRNS('Midia.aspx?tipo=SIN&id="+idsin+"&Alterar=1&Session=SIN', 800, 550, 1);};")
  except:
    driver_object.execute_script("document.getElementsByClassName('AtalhoSequencial noBr')[document.getElementsByClassName('AtalhoSequencial noBr').length-1].click()")

  if idsin==0:
    idsin=''
  else:
    idsin = str(idsin)
  hwnd=driver_object.current_window_handle
  Switch_By_Partial_Url(driver_object,['Midia.aspx',idsin])

  script="__doPostBack('ctl00$Body$dlMidias$ctl01$Linkbutton1','')"
  Timer(3)
  driver_object.execute_script(script)

  if str(type(midia)).lower()=="list":
    for path in midia:
      driver_object.find_element(By.ID,'file').send_keys(path)
      Timer(2)
      print("Texto:",driver_object.find_element(By.ID,'file').text)
      driver_object.execute_script("document.getElementById('cmdSalvar').click()")
      Timer(2)
      driver_object.execute_script("javascript:__doPostBack('ctl00$Body$dlMidias$ctl02$Linkbutton1','')")
      Timer(1)

  else:
    driver_object.find_element(By.ID,'file').send_keys(midia)
    Timer(2)
    print("Texto:",driver_object.find_element(By.ID,'file').text)
    driver_object.execute_script("document.getElementById('cmdSalvar').click()")
    Timer(1)


  driver_object.switch_to.window(hwnd)


def Ending(driver_object):
  Timer(2)
  print("Finalizando ...")
  driver_object.execute_script("document.getElementById('butAcao2').click()")
  Timer(1)
  driver_object.execute_script("document.getElementById('butContinuar').click()")
  Timer(1)
  idsin = driver_object.find_element(By.ID,"label_numeroSin").text
  idsin = idsin.split(":")[1].strip()
  driver_object.execute_script("document.getElementById('butSim').click()")
  Timer(3)
  driver_object.execute_script("__doPostBack('ctl00$Body$TopMenu1$dlmenu$ctl00$lbutopcao','')")
  return idsin
  print("Processo concluído!")

def Klassmatt(driver_object,title='',description='',txtTipo='',txtUnidadeMedida='',urgent=False,grupo='',subgrupo='',condicao_transacao='',midia=''):
  Start_Registration(driver_object,title,description)
  Set_Types(driver_object,txtTipo,txtUnidadeMedida)
  Set_AS_Urgent(driver_object,urgent)
  Set_Classification(driver_object,grupo,subgrupo,condicao_transacao)
  Set_Midia(driver_object,midia)
  idsin= Ending(driver_object)
  return idsin

# **Menu Principal**

In [125]:
#@title Login e Senha:
user_login = 'francisco.sousa' #@param {type:"string"}
#input("Digite o login\n")
password_login = 'Fje$1321' #@param {type:"string"}
#input("Digite a senha\n")

In [126]:
#@title Parâmetros:
codigo_email='' #@param {type:"string"}

caminho_midia=r"/content/drive/MyDrive/MATERIAIS/Midias/Provision Cranes.pdf" #@param {type:"string"}

# **Executar**

In [127]:
driver=Create_Driver()
Go(driver)
Login(driver)
title="CHAVE LIMITADORA DO IÇAMENTO (SUPERIOR)"
description = "CHAVE LIMITADORA DO IÇAMENTO (SUPERIOR) - PN:P74-13 - FABRI:DREGGEN CRANE AS - EMBAR:MBEL - EQUIP:PROVISION CRANE - DISPOSITIVO DE SEGURANÇA DA LANÇA PARA IÇAR (SUPERIOR) - PN:P74-13 - FABRICANTEDREGGEN CRANE AS - EMBARCACAO:MBEL - EQUIPAMENTO:PROVISION CRANE - MODELO:DK 160-10T-12M - SERIE:L677A-L - NOME INGLES:LIMIT SWITCH, HOIST TOP POS. - APLICACAO:SEGURANÇA/ BELMONTE III EQUIPAMENTO PORTUGUES:GUINDASTE DE PROVISÕES - MATERIAL: - OBS:"
grupo="58. MATERIAIS MECANICA (EQUIPAMENTOS / SOBRESSALENTES)"
subgrupo="195. GUINDASTE PARA CONTAINER ESTACIONARIOS (EQUIPAMENTOS, ACESSORIOS E SOBRESSALENTES)"
cond_transacao="1. USO E CONSUMO"
Klassmatt(driver,title,description,'MATERIAL','PECA',True,grupo,subgrupo,cond_transacao,caminho_midia)
#print(driver.page_source)

Start Registration ...
Start Setting Types ...
Verifing Urgencie ...
Setting classification ...
setting midia up ...
Texto: 
Finalizando ...


'123907'